E' stata creata un unica API Getway con due endpoint:
-createUser
-getuserbyID

https://5yi8g28lu8.execute-api.us-east-1.amazonaws.com

## <span style="color:red">ADDUSER</span>

**lambda function**

In [118]:
from boto3.dynamodb.types import TypeDeserializer

In [119]:
a = TypeDeserializer()

In [ ]:
import json
import boto3
from boto3.dynamodb.types import TypeSerializer
import uuid

dynamodb = boto3.client('dynamodb') ## ogni servizio che voglio utilizzare devo crea un istanza di client
ser = TypeSerializer()
TABLENAME = "Users"

class user:
    def __init__(self,body):
        self.msg = ""
        self.isValid = False
        res,msg = self.__check_variable(body,"username",str)
        if res:
            self.body = body
            self.body["ID"] = self.__generate_ID()
            self.username = msg
            self.isValid = True
        else:
            self.msg = msg
            
    def __check_variable(self,body,prop_name,prop_type):
        if not prop_name in body.keys():
            return False,f"{prop_name}  not present in request"
        username = body[prop_name]
        if not isinstance(username,prop_type):
            print("SORETA")
            return False,f"{prop_name} is not of type {prop_type}"
        return True,username
    
    def __generate_ID(self):
        return uuid.uuid4().hex
    
    def send_request(self,ser:boto3.dynamodb.types.TypeSerializer,dynamodb,tableName):
        if self.isValid:
            req = ser.serialize(self.body)["M"]
            try:
                dynamodb.put_item(TableName=tableName, Item=req)
                self.msg = self.body["ID"]
            except Exception as e:
                self.msg = "errore nell inserimento dello user {}: {}".format(self.username,e)


def lambda_handler(event, context):
    # TODO implement
    messages = []
    req = event["body"]
    if isinstance(req,list):
        for user_info in req:
            print(user_info)
            usr =  user(user_info)
            usr.send_request(ser,dynamodb,TABLENAME)
            messages.append(usr.msg)
    else:
        messages.append("il body deve contenere un array di utenti")
        
    if len(messages )== 0:
        statusCode = 200
        body = "ok"
    else:
        statusCode = 400
        body = messages
    return {
        'statusCode': statusCode,
        'body': json.dumps(body)
}
    
    


**esempio chiamata**

In [114]:
url = "https://5yi8g28lu8.execute-api.us-east-1.amazonaws.com/createUser"

la chiamata è fattibile in POST

In [115]:
data = [
    {
    "username":"pasq47",
    "nome":"ciccio",
    "cognome":"pincopallino"
    },
    {
    "cognome":"stefannonia"
    },
    {
    "username":55,
    "cognome":"stefannonia"
    },
]

In [116]:
res = requests.post(url,json=data)

In [117]:
print(res.json())

['bfaf6e9602684016b81b071a526bc683', 'username  not present in request', "user 55 is not of type <class 'str'>"]


## <span style="color:red">GETUSERBYID</span>

**lambda**

In [ ]:
import json
import boto3
from boto3.dynamodb.types import TypeDeserializer

dynamoDB = boto3.client("dynamodb")
TABLENAME = "Users"
def lambda_handler(event, context):
    req = event["queryStringParameters"]
    if not "ID" in req.keys():
        return {
            'statusCode': 200,
            'body': json.dumps("no ID in request")
        }
    ID =  req["ID"]
    ID = "dd6e857c782448a8811289e5853a73e0"
    if isinstance(ID,str):
        res = dynamoDB.get_item(
            TableName=TABLENAME,
            Key={
                'ID': {
                  'S': ID
                }
            }
          )
        if "Item" in res.keys():
            item = res["Item"]
            des = TypeDeserializer()
            print(item)
            body = {k: des.deserialize(v) for k,v in item.items()}
        else:
            body = "user not found"
    else:
        body = "ID is not fo type string"
         
    return {
        'statusCode': 200,
        'body': json.dumps(body)
    }


**esempio chiamata**

la chiamata è in get

In [191]:
url = "https://5yi8g28lu8.execute-api.us-east-1.amazonaws.com/getUserById"

In [270]:
params = {"ID":"dd6e857c782448a8811289e5853a73e0"}
requests.get(url,params=params).json()

{'username': 'pasq47',
 'nome': 'ciccio',
 'ID': 'dd6e857c782448a8811289e5853a73e0'}

In [271]:
params = {"ID":"false"}
requests.get(url,params=params).json()

'user not found'